In [1]:
import cv2, time, pandas
from datetime import datetime


In [3]:
first_frame=None
status_list=[None,None] # list used the store the status of the object entered and exit
times=[]# list to date time of an event
df=pandas.DataFrame(columns=["Start","End"])

In [4]:
video=cv2.VideoCapture(0)

In [6]:
while True:
    check, frame = video.read() # capture the first frame 
    status=0 # used to check weather the object enter the screen
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) # convert to gray
    gray=cv2.GaussianBlur(gray,(21,21),0)#(21,21)width and height of gaussian no,0 is standard deviation

    if first_frame is None:
        first_frame=gray
        continue

    delta_frame=cv2.absdiff(first_frame,gray) # claculate the difference between first and the current frame
    thresh_frame=cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]# 255 is the color ,30 is the height of delta frame
    thresh_frame=cv2.dilate(thresh_frame, None, iterations=2)# clearity valye more iteration more clear

    (_,cnts,_)=cv2.findContours(thresh_frame.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # create the rectangle box on the object or contore

    for contour in cnts: # size of circle
        if cv2.contourArea(contour) < 10000: 
            continue
        status=1

        (x, y, w, h)=cv2.boundingRect(contour) # full screen rectangle
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 3)#(0,255,0)color of rectangle, 3 is the width
    status_list.append(status)# insert the status in status_append

    status_list=status_list[-2:]


    if status_list[-1]==1 and status_list[-2]==0: # checking the status of last  and  second last of list weather it have change or not 
        times.append(datetime.now()) # put current time in yhe list time
    if status_list[-1]==0 and status_list[-2]==1:
        times.append(datetime.now()) # put current time in yhe list time


    cv2.imshow("Gray Frame",gray)
    cv2.imshow("Delta Frame",delta_frame)
    cv2.imshow("Threshold Frame",thresh_frame)
    cv2.imshow("Color Frame",frame)

    key=cv2.waitKey(1)

    if key==ord('q'): # key to quit the app
        if status==1:
            times.append(datetime.now()) # record the current time if object present
        break

print(status_list)
print(times)

for i in range(0,len(times),2): #start from o to the len(time) with 2 step  
    df=df.append({"Start":times[i],"End":times[i+1]},ignore_index=True) # zero time in 1 column(i), 1 time in second column(i+1) and repead till end of the list

df.to_csv("Times.csv")

video.release()
cv2.destroyAllWindows


error: OpenCV(3.4.2) c:\projects\opencv-python\opencv\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'
